In [1]:
import pandas as pd
import sql_functions as sql_int

In [2]:
tournament_result = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/prismatic_Legends_OQ1.csv',
                                converters={'playerName': str.strip})

In [3]:
df_tournament_result = tournament_result.copy()
tournament_result

,playerName,placement,group,round,tournament
0,Loescher,2,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
1,Bufferingo,3,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
2,GrexGoesFlex,1,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
3,Serphax,4,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
4,Vahrulv,5,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
...,...,...,...,...,...
508,Hamhamgang,7,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
509,Dansky,8,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
510,V9 Poseidon,6,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
511,FNCSkya1,3,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...


#### changing points into placement

### Testing names are real

In [4]:
def check_new(df_new,df_exist,columns):
    df_new = df_new.copy()
    df_exist = df_exist.copy()
    df_exist['already_exist'] = 'Yes'
    drop_columns = df_exist.columns.difference(df_new.columns)
    df_concat = pd.concat([df_exist,df_new],ignore_index=True)
    df_concat = (df_concat
                 .drop_duplicates(subset=[columns],keep=False)
                 .query('already_exist != "Yes"')
                 .drop(columns=drop_columns)
    )
    return df_concat

In [5]:
df_player = tournament_result[['playerName']].copy().drop_duplicates()

schema = 'tft_table'
table = 'alternative_name'
sql_query = f'SELECT player_id,player_name FROM {schema}.{table};'
alternative_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['player_id','player_name'])
df_alternative_name = alternative_name.copy()
df_player = df_player.drop_duplicates().rename(columns={'playerName': 'import_name' })
df_player_import = check_new(df_player,
                             df_alternative_name.rename(columns={'player_name' :'import_name' }),
                             'import_name')
df_player_import

,import_name


### aDDING STUFF

In [6]:
df_tournament_result = df_tournament_result.dropna()
if 'placement' in df_tournament_result.columns: 
    pass 
else: 
    df_tournament_result['placement'] = df_tournament_result['points'].apply(lambda x: 9-int(x))
    df_tournament_result = df_tournament_result.drop(columns='points')
           
df_tournament_result[df_tournament_result.duplicated(keep= False, subset=['tournament','group','round','placement']) == True].sort_values(by=['tournament','round','group'])

,playerName,placement,group,round,tournament


In [7]:
df_tournament_result = df_tournament_result.drop_duplicates()

#### create Result tables

In [8]:
df_tournament_result = df_tournament_result.drop_duplicates()


In [9]:
df_tournament_result

,playerName,placement,group,round,tournament
0,Loescher,2,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
1,Bufferingo,3,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
2,GrexGoesFlex,1,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
3,Serphax,4,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
4,Vahrulv,5,A,1,A1 eSports Prismatic Legends: Season 1 Open Qu...
...,...,...,...,...,...
508,Hamhamgang,7,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
509,Dansky,8,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
510,V9 Poseidon,6,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...
511,FNCSkya1,3,BS,6,A1 eSports Prismatic Legends: Season 1 Open Qu...


In [10]:

df_lobby_result = (pd.pivot_table(df_tournament_result,values='playerName',columns='placement',index=['tournament','round','group'],aggfunc='first')
                   .reset_index()
                   .rename(columns={1:'winner',2:'runnerup',3:'third',4:'fourth',5:'fifth',6:'sixth',7:'seventh',8:'eighth'})
                   .drop(columns=[9],errors='ignore')
                   )
df_lobby_result

placement,tournament,round,group,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth
0,A1 eSports Prismatic Legends: Season 1 Open Qu...,1,A,GrexGoesFlex,Loescher,Bufferingo,Serphax,Vahrulv,NekoGirlLaura,NaN,NaN
1,A1 eSports Prismatic Legends: Season 1 Open Qu...,1,AA,wandherker,crazynike,Rocklho,Tryndaxxx,Duckyy,Fr1day,NaN,NaN
2,A1 eSports Prismatic Legends: Season 1 Open Qu...,1,AB,TFTobyy,Mernio,Fridoetending,Michaelas01,Germorad,towed,NaN,NaN
3,A1 eSports Prismatic Legends: Season 1 Open Qu...,1,AC,Raki,Lorus,Wackentrooper,kiserlodski,AR Drip,Alexath,NaN,NaN
4,A1 eSports Prismatic Legends: Season 1 Open Qu...,1,AD,Iphora,DÄÜP,TFTobby,Beoorc,Saaschtin,TFT Zenek,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
67,A1 eSports Prismatic Legends: Season 1 Open Qu...,5,BS,GoeksuLive,V9 Poseidon,FNCSkya1,neilOoO,silasdvTFT#EUW,Hamhamgang,Dansky,GrexGoesFlex
68,A1 eSports Prismatic Legends: Season 1 Open Qu...,6,BP,Puttyx,Kuro Just Hit,Loescher,gluteus maximus,shooku,br0ken99,Dacano,Sunny#1412
69,A1 eSports Prismatic Legends: Season 1 Open Qu...,6,BQ,h4udrauf,TFTMarx,keaded,mofred,leinadsaibot29,Cyuaa,Shantao,Fridoetending
70,A1 eSports Prismatic Legends: Season 1 Open Qu...,6,BR,Prologie,Bäckchen,Steachey,Mr Slugma,Calamity,phinphinthao,TFTobyy,alifron


#### Take tournament ID's

In [11]:
schema = 'tft_table'
table = 'tournament'
sql_query = f'''select name, tournament_id
                FROM {schema}.{table};
                '''

df_tounament_id = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['tournament','tournament_id'])

df_tounament_push = (pd.merge(df_lobby_result,df_tounament_id, how='left',left_on='tournament',right_on='tournament')
                     .drop(columns='tournament')
                     )

#### Checking tournament result already exists

In [12]:
table = 'lobby'
sql_query = f'''SELECT tournament_id,round,party
                FROM {schema}.{table};
                '''
df_existing_lobby = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['tournament_id','round','group'])
df_existing_lobby['existing'] = 'Yes'
df_no_dups = pd.concat([df_tounament_push,df_existing_lobby],ignore_index=True)
df_no_dups = df_no_dups.drop_duplicates(subset=['tournament_id','group','round'],keep=False).rename(columns={'group': 'party'})

df_no_dups = (df_no_dups
                  .drop_duplicates(subset=['tournament_id','round','party'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['existing'],axis=1)
                  )

In [13]:
df_no_dups

,round,party,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth,tournament_id
0,1,A,GrexGoesFlex,Loescher,Bufferingo,Serphax,Vahrulv,NekoGirlLaura,NaN,NaN,28
1,1,AA,wandherker,crazynike,Rocklho,Tryndaxxx,Duckyy,Fr1day,NaN,NaN,28
2,1,AB,TFTobyy,Mernio,Fridoetending,Michaelas01,Germorad,towed,NaN,NaN,28
3,1,AC,Raki,Lorus,Wackentrooper,kiserlodski,AR Drip,Alexath,NaN,NaN,28
4,1,AD,Iphora,DÄÜP,TFTobby,Beoorc,Saaschtin,TFT Zenek,NaN,NaN,28
...,...,...,...,...,...,...,...,...,...,...,...
67,5,BS,GoeksuLive,V9 Poseidon,FNCSkya1,neilOoO,silasdvTFT#EUW,Hamhamgang,Dansky,GrexGoesFlex,28
68,6,BP,Puttyx,Kuro Just Hit,Loescher,gluteus maximus,shooku,br0ken99,Dacano,Sunny#1412,28
69,6,BQ,h4udrauf,TFTMarx,keaded,mofred,leinadsaibot29,Cyuaa,Shantao,Fridoetending,28
70,6,BR,Prologie,Bäckchen,Steachey,Mr Slugma,Calamity,phinphinthao,TFTobyy,alifron,28


In [14]:
sql_int.push_to_database(df_no_dups,table,sql_int.get_engine_alchemy(),schema)

The lobby table was imported successfully.
